In [67]:
import pandas as pd
from utils import io
from utils import ts_utils
import importlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy import sqrt
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.stattools import adfuller, pacf

get_ipython().run_line_magic('matplotlib', 'inline')
import warnings
warnings.filterwarnings('ignore')



In [69]:
importlib.reload(io)
importlib.reload(ts_utils)
data_handler = io.DataHandler(data_path = 'data')

bs_areas = data_handler.get_bussines_areas()
# bs_areas = ['EQ']
# bs_areas = ['Consumables']

regions = data_handler.get_regions()
# regions = [33]
# regions = [56]

exog_features = [
    'MGDP_Index',
    'EBIT_Index',
    'Revenue_Index',
    'fleet1',
    'fleet2',
    'fleet3',
    'Copper',
    'Gold',
    'Iron Ore',
    'Nickel',
    'Silver',
    'Zinc',
]

all_models = True

for bs_area in bs_areas:
    for region in regions: 
        
        if bs_area == 'Consumables' and region == 56:
            continue
        
        name = f'{bs_area}_{region}'
        full_name = f'{bs_area} {region}'

        sales_bus_region = data_handler.get_sales_per_business_area_and_region(business_area = bs_area,
                                                                               region = region)
        
        sales_bus_region = data_handler.add_monthly_financial_info(sales_df = sales_bus_region,
                                                                   region = region)

        ts_handler = ts_utils.TimeSeriesHandler(name = full_name,
                                                df = sales_bus_region,
                                                # endog = 'sales_value')
                                                endog = 'sales_value', exog = exog_features)
        # if len(ts_handler.test) < 5:
        #     print('TEST NOT FULL ' + full_name)
        #     print(len(ts_handler.test))

        exog_forecast = pd.DataFrame()
        for exog in ts_handler.exog:
            # print(exog)
            exog_name = f'exog_{exog}'
            exog_ts_handler = ts_utils.TimeSeriesHandler(name = exog_name,
                                                         df = sales_bus_region,
                                                         endog = exog,
                                                         exog = None)
            # if len(exog_ts_handler.test) < 5:
            #     print('TEST NOT FULL '+ full_name + ' ' + exog_name)
            #     print(len(exog_ts_handler.test))
            exog_ts_handler.determine_orders()
            exog_ts_handler.fit_sarimax_exog(plot = False if all_models else True)
            exog_forecast[exog] = exog_ts_handler.forecast

        ts_handler.set_exog_forecast(exog_forecast)

        ts_handler.determine_orders()

        ts_handler.fit_sarimax(name = full_name,
                               plot_fit = True,
                               plot_prediction = True,
                               rolling_prediction = True,
                               plot_rolling = True)
        
        
        if all_models: break
    if all_models: break


MGDP_Index
EBIT_Index
Revenue_Index
fleet1
fleet2
fleet3
Copper
Gold
Iron Ore
Nickel
Silver
Zinc


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x000001D5BA9E49D0>>
Traceback (most recent call last):
  File "D:\python_projects\ignize_final\venv\Lib\site-packages\ipykernel\ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 

KeyboardInterrupt

